In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/test-data/Social.pdf
/kaggle/input/subject/economics/Complete Book Class 10 Economics.pdf
/kaggle/input/subject/maths/NCERT Maths Class 12 Book (Part I).pdf
/kaggle/input/subject/maths/NCERT Class 11 Maths Book.pdf
/kaggle/input/subject/maths/NCERT Class 12 Maths Book (Part II).pdf
/kaggle/input/subject/maths/NCERT Book Maths Class 9.pdf
/kaggle/input/subject/maths/Complete Class 10 NCERT Maths Book 2023-24.pdf
/kaggle/input/subject/History/NCERT Book Class 9 History.pdf
/kaggle/input/subject/History/Complete Book Class 10 History (1).pdf
/kaggle/input/subject/english/X English First Flight Complete Book 2023-24.pdf
/kaggle/input/subject/english/NCERT Class 12 English Flamingo Book.pdf
/kaggle/input/subject/english/Complete Class 11 English Hornbill Book.pdf
/kaggle/input/subject/english/NCERT Book Class 9 English Beehive.pdf
/kaggle/input/subject/political science/Complete Book Class 10 Political Science.pdf
/kaggle/input/subject/chemistry/NCERT Class 11 Chemistry Book (

In [2]:
!pip install chardet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.4/199.4 kB 7.3 MB/s eta 0:00:00


In [3]:
!pip install pymu

  Preparing metadata (setup.py) ... done
  Created wheel for pymu: filename=PyMU-0.2.23-py3-none-any.whl size=17515 sha256=c473ebbc27b5109f9894151c2996d57edacc35465e280e6dc38158ae53944875
  Stored in directory: /root/.cache/pip/wheels/7c/cd/d2/19bb9cbcc10cab3edf92875a568080a03745102d7f08f5fb09
Successfully built pymu


In [6]:
!pip install PyMuPdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 36.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 30.3 MB/s eta 0:00:0000:0100:01


In [65]:
import os
import chardet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
import fitz

In [10]:
# Step 1: Prepare the dataset
root_dir = '/kaggle/input/subject'  # Update this with the correct path to your 'subject' directory
categories = ['History', 'chemistry', 'economics', 'english', 'enterainment', 'geography', 'maths', 'physics', 'political science', 'science']

In [14]:
# Step 2: Load and preprocess the text data
documents = []
labels = []

for i, category in enumerate(categories):
    folder_path = os.path.join(root_dir, category)
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            # Read PDF content using PyMuPDF (fitz)
            with fitz.open(file_path) as pdf_document:
                text = ''
                for page_num in range(pdf_document.page_count):
                    page = pdf_document[page_num]
                    text += page.get_text()
                documents.append(text)
                labels.append(i)
        except Exception as e:
            print(f"Error reading file: {file_path}. Skipping this file. Error: {e}")

In [15]:
# Step 3: Train the classifier
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  
X_tfidf = tfidf_vectorizer.fit_transform(documents)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(labels)

classifier = MultinomialNB()
classifier.fit(X_tfidf, y_encoded)

MultinomialNB()

In [74]:
# Define the number of folds for cross-validation
from sklearn.model_selection import KFold
kf = KFold(n_splits=10, shuffle=True, random_state=42)
accuracies = []
for train_idx, test_idx in kf.split(X_tfidf, y_encoded):
    X_train_fold, X_test_fold = X_tfidf[train_idx], X_tfidf[test_idx]
    y_train_fold, y_test_fold = y_encoded[train_idx], y_encoded[test_idx]
    
    classifier = MultinomialNB()
    classifier.fit(X_train_fold, y_train_fold)
    
    y_pred_fold = classifier.predict(X_test_fold)
    accuracy_fold = accuracy_score(y_test_fold, y_pred_fold)
    fold_accuracy_scores.append(accuracy_fold)

# Calculate and print average accuracy across folds
average_accuracy = sum(fold_accuracy_scores) / num_folds
print(f"AVERAGE ACCURACY ACROSS {num_folds} FOLDS: {average_accuracy:.2f}")

AVERAGE ACCURACY ACROSS 5 FOLDS: 2.60


In [52]:
# Step 4: Use the trained model for inference
def classify_document(file_path):
    try:
        with fitz.open(file_path) as pdf_document:
            text = ""
            for page_num in range(pdf_document.page_count):
                page = pdf_document[page_num]
                text += page.get_text()
        
        # Preprocess the text and predict category with Laplace smoothing
        text_tfidf = tfidf_vectorizer.transform([text])
        predicted_category_encoded = classifier.predict(text_tfidf)[0]  # Access the first element directly
        
        # Check if the predicted category code is within the expected range
        if 0 <= predicted_category_encoded < len(categories):
            predicted_category = label_encoder.classes_[predicted_category_encoded]
        else:
            predicted_category = "Unknown"
        
        return predicted_category, predicted_category_encoded
    except Exception as e:
        return f"Error classifying the document. Error: {e}"


In [53]:
category_mapping = {i: category for i, category in enumerate(categories)}
print("Category Mapping:")
for code, category_name in category_mapping.items():
    print(f"Numeric Code: {code}, Category: {category_name}")

Category Mapping:
Numeric Code: 0, Category: History
Numeric Code: 1, Category: chemistry
Numeric Code: 2, Category: economics
Numeric Code: 3, Category: english
Numeric Code: 4, Category: enterainment
Numeric Code: 5, Category: geography
Numeric Code: 6, Category: maths
Numeric Code: 7, Category: physics
Numeric Code: 8, Category: political science
Numeric Code: 9, Category: science


In [80]:
import os
test_data_folder = '/kaggle/input/tests-data'
test_files = [os.path.join(test_data_folder, file) for file in os.listdir(test_data_folder) if file.endswith('.pdf')]

# Loop through each test file and predict its category
for test_document_path in test_files:
    predicted_category, predicted_category_encoded = classify_document(test_document_path)
    predicted_name = category_names[predicted_category]
    
    print(f"Predicted Category Name: {predicted_name}")
    print(f"Predicted Category: {predicted_category}")
    print("------")

Predicted Category Name: maths
Predicted Category: 6
------
Predicted Category Name: maths
Predicted Category: 6
------
Predicted Category Name: maths
Predicted Category: 6
------
